In [ ]:
def upload_files(contents, filenames, current_data):
    encoded_files = current_data[0] if current_data else []
    file_contents = current_data[1] if current_data else []
    file_names = current_data[2] if current_data else []

    if contents is not None:
        for content, filename in zip(contents, filenames):
            try:
                # Split content string to get the base64 part
                content_type, content_string = content.split(',')
                
                # Decode base64
                try:
                    decoded = base64.b64decode(content_string)
                except Exception as e:
                    print(f"Error decoding base64: {e}")
                    continue

                if 'pdf' in filename.lower():
                    print(f"Processing PDF: {filename}")
                    try:
                        # First try with PyPDF2
                        pdf = io.BytesIO(decoded)
                        reader = PdfReader(pdf)
                        text_content = []
                        for page in reader.pages:
                            text_content.append(page.extract_text())
                        
                        # Store the text content
                        file_contents.append('\n'.join(text_content))
                        # Store the original PDF
                        encoded_files.append(f'data:application/pdf;base64,{content_string}')
                        file_names.append(filename)
                        print(f"Successfully processed PDF {filename}")
                        
                    except Exception as pdf_error:
                        print(f"Error processing PDF with PyPDF2: {pdf_error}")
                        # If PyPDF2 fails, try with pdfplumber
                        try:
                            pdf = io.BytesIO(decoded)
                            with pdfplumber.open(pdf) as plumber_pdf:
                                text_content = []
                                for page in plumber_pdf.pages:
                                    text = page.extract_text()
                                    if text:
                                        text_content.append(text)
                            
                            file_contents.append('\n'.join(text_content))
                            encoded_files.append(f'data:application/pdf;base64,{content_string}')
                            file_names.append(filename)
                            print(f"Successfully processed PDF {filename} with pdfplumber")
                            
                        except Exception as plumber_error:
                            print(f"Both PDF readers failed. PyPDF2 error: {pdf_error}, Pdfplumber error: {plumber_error}")
                            continue

                elif 'txt' in filename.lower():
                    # Handle text files
                    try:
                        text_content = decoded.decode('utf-8')
                        encoded_files.append(f'data:text/plain;charset=utf-8,{content_string}')
                        file_contents.append(text_content)
                        file_names.append(filename)
                    except Exception as e:
                        print(f"Error processing text file: {e}")
                        continue

            except Exception as e:
                print(f"Error processing file {filename}: {e}")
                continue

    return encoded_files, file_contents, file_names